In [1]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

In [2]:
page_url = 'https://www.scrapethissite.com/pages/frames/?frame=i'
base_url = 'https://www.scrapethissite.com'

In [3]:
response = requests.get(page_url)
if response.status_code == 200:
    print("Access granted to the page.")
    soup = BeautifulSoup(response.content, 'html.parser')
else:
    print(f"Failed to access the page. Status code: {response.status_code}")
    exit()

turtles = soup.find_all('div', class_='col-md-4 turtle-family-card')

Access granted to the page.


In [4]:
turtles_data = {
    "Name": [],
    "Known_As": [],
    "Discovery_Year": [],
    "Discovered_By": []
}

for turtle in turtles:
    name_tag = turtle.find('h3', class_='family-name')
    if name_tag:
        name = name_tag.text.strip()
        turtles_data['Name'].append(name)
    else:
        turtles_data['Name'].append(None)

    link_tag = turtle.find('a', href=True)
    if link_tag:
        detail_url = base_url + link_tag['href']
        detail_response = requests.get(detail_url)
        detail_soup = BeautifulSoup(detail_response.content, 'html.parser')
        info = detail_soup.find('p', class_='lead')

        if info:
            known_tag = info.find('strong', class_='common-name')
            known_as = known_tag.text.strip() if known_tag else None
            turtles_data['Known_As'].append(known_as)

            year_match = re.search(r'\b(1[7-9]\d{2}|20\d{2})\b', info.text)
            year = year_match.group(0) if year_match else None
            turtles_data['Discovery_Year'].append(year)

            discoverer_match = re.search(r'by ([A-Z][a-zA-Z\s\.\-]*)', info.text)
            discoverer = discoverer_match.group(1).strip() if discoverer_match else None
            turtles_data['Discovered_By'].append(discoverer)
        else:
            turtles_data['Known_As'].append(None)
            turtles_data['Discovery_Year'].append(None)
            turtles_data['Discovered_By'].append(None)
    else:
        turtles_data['Known_As'].append(None)
        turtles_data['Discovery_Year'].append(None)
        turtles_data['Discovered_By'].append(None)

print(turtles_data)

{'Name': ['Carettochelyidae', 'Cheloniidae', 'Chelydridae', 'Dermatemydidae', 'Dermochelyidae', 'Emydidae', 'Geoemydidae', 'Kinosternidae', 'Platysternidae', 'Testudinidae', 'Trionychidae', 'Chelidae', 'Pelomedusidae', 'Podocnemididae'], 'Known_As': ['Pig-nosed turtle', 'Sea turtles', 'Snapping turtles', 'Central American river turtle', 'Leatherback sea turtle', 'Pond or water turtles', 'Asian river, leaf, roofed or Asian box turtles', 'Mud or musk turtles', 'Big-headed turtle', 'Tortoises', 'Softshell turtles', 'Austro-American sideneck turtles', 'Afro-American sideneck turtles', 'Madagascar big-headed, Big-headed Amazon River turtle and South American sideneck river turtles'], 'Discovery_Year': ['1887', '1811', '1831', '1870', '1843', '1815', '1868', '1857', '1869', '1788', '1826', '1831', '1868', '1869'], 'Discovered_By': ['Boulenger.', 'Oppel.', 'Gray.', 'Gray.', 'Fitzinger.', 'Rafinesque.', 'Theobald.', 'Agassiz.', 'Gray.', 'Batsch.', 'Fitzinger.', 'Gray.', 'Cope.', 'Gray.']}


In [5]:
df = pd.DataFrame(turtles_data)
df

,Name,Known_As,Discovery_Year,Discovered_By
0,Carettochelyidae,Pig-nosed turtle,1887,Boulenger.
1,Cheloniidae,Sea turtles,1811,Oppel.
2,Chelydridae,Snapping turtles,1831,Gray.
3,Dermatemydidae,Central American river turtle,1870,Gray.
4,Dermochelyidae,Leatherback sea turtle,1843,Fitzinger.
5,Emydidae,Pond or water turtles,1815,Rafinesque.
6,Geoemydidae,"Asian river, leaf, roofed or Asian box turtles",1868,Theobald.
7,Kinosternidae,Mud or musk turtles,1857,Agassiz.
8,Platysternidae,Big-headed turtle,1869,Gray.
9,Testudinidae,Tortoises,1788,Batsch.


In [6]:
df.to_csv('turtles_data.csv', index=False)

print("Data saved to turtles_data.csv")

Data saved to turtles_data.csv
